In [1]:
import numpy as np
from time import time
from tqdm import tqdm

import os
if 'HOSTNAME' not in os.environ:
    os.environ['HOSTNAME'] = 'ODIN' # or 'ODIN'ArithmeticError
os.makedirs('output', exist_ok=True) # check that output dir is there

from core.insilico_exps import ExperimentEvolution
from core.GAN_utils import upconvGAN
from core.CNN_scorers import TorchScorer
from core.Optimizers import Genetic, CholeskyCMAES

import torch
from torchvision import transforms as pth_transforms


image_size = (224,224)

arch = 'vit_base'
patch_size = 16

In [2]:
# import dino_vit as vits


# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = 'cpu'
#     # build model
# import dino_vit as vits
# model = vits.__dict__[arch](patch_size=patch_size, num_classes=0)
# for p in model.parameters():
#     p.requires_grad = False
# model.eval()
# model.to(device)


# if arch == "vit_small" and patch_size == 16:
#     url = "dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth"
# elif arch == "vit_small" and patch_size == 8:
#     url = "dino_deitsmall8_300ep_pretrain/dino_deitsmall8_300ep_pretrain.pth"  # model used for visualizations in our paper
# elif arch == "vit_base" and patch_size == 16:
#     url = "dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth"
# elif arch == "vit_base" and patch_size == 8:
#     url = "dino_vitbase8_pretrain/dino_vitbase8_pretrain.pth"
# if url is not None:
#     print("Since no pretrained weights have been provided, we load the reference pretrained DINO weights.")
#     state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + url)
#     model.load_state_dict(state_dict, strict=True)
# else:
#     print("There is no reference weights available for this model => We use random weights.")

In [3]:
transform = pth_transforms.Compose([
    pth_transforms.Resize(image_size),
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [7]:
tmpsavedir = 'output' # Temporary save directory
device = 'cpu'
# load optimizer
optim = CholeskyCMAES(4096, population_size=40, init_sigma=2.0, Aupdate_freq=10, init_code=np.zeros([1, 4096]))
# un-comment to use our new one! 
explabel = "dino_vit"
model_unit =  ('dino:' + arch + ':' + str(patch_size), '.blocks.0.attn.Dropoutattn_drop', 0, 0, 0)
Exp = ExperimentEvolution(model_unit, savedir=tmpsavedir, explabel=explabel, optimizer=optim, max_step=500, imgsize=image_size, device=device)


Space dimension: 4096, Population size: 40, Select size:20, Optimization Parameters:
Initial sigma: 2.000
cc=0.001, cs=0.050, c1=0.000 damps=1.050
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.


In [8]:
# run evolutions
t1 = time()
Exp.run(optim.get_init_pop())
t2 = time()
print(t2 - t1, "sec")  
Exp.visualize_best()
Exp.visualize_trajectory()
Exp.save_last_gen()

synth img scores: mean 0.129 +- std 0.000
step 0  time: total 0.46s | GAN visualize 0.20s   CNN score 0.12s   optimizer step 0.15s
sigma: 1.94
synth img scores: mean 0.209 +- std 0.029
step 1  time: total 3.74s | GAN visualize 0.10s   CNN score 3.58s   optimizer step 0.06s
sigma: 1.88
synth img scores: mean 0.253 +- std 0.040
step 2  time: total 3.48s | GAN visualize 0.07s   CNN score 3.34s   optimizer step 0.07s
sigma: 1.84
synth img scores: mean 0.318 +- std 0.043
step 3  time: total 2.97s | GAN visualize 0.07s   CNN score 2.83s   optimizer step 0.07s
sigma: 1.81
synth img scores: mean 0.352 +- std 0.051
step 4  time: total 2.89s | GAN visualize 0.11s   CNN score 2.71s   optimizer step 0.07s
sigma: 1.78
synth img scores: mean 0.428 +- std 0.075
step 5  time: total 3.34s | GAN visualize 0.07s   CNN score 3.19s   optimizer step 0.08s
sigma: 1.75
synth img scores: mean 0.523 +- std 0.076
step 6  time: total 2.92s | GAN visualize 0.08s   CNN score 2.76s   optimizer step 0.08s
sigma: 1.73